# magicpatch demos

## Still runs normal JavaScript

In [1]:
var n = "bob";
console.log("hi", n);

hi bob


## %magic example
Magicpatch adds [%magic commands](https://ipython.readthedocs.io/en/stable/interactive/magics.html) to IJavascript. Magics are intended to make development faster and easier. Here's a magic called %echo that does... pretty much what you would expect:

In [2]:
%echo hi there

hi there


## %addmagic

You can also add your own magics. The syntax is `%addcmd cmdname fnname`.  `cmdname` must start with a symbol like `%` or `.`, and `fnname` is a string that is the name of a function. `fnname` is not checked to see if it exists until the magic is called. If you want to create a npm module for adding magics, you can use the global method `$$.addMagic()`, which has the same syntax.

In [3]:
function bar() {
    console.log("LETS ALL GO TO THE BAR");
}
%addmagic %foo bar

[ added magic: '%foo' which will call function 'bar' ]


In [4]:
%foo

LETS ALL GO TO THE BAR


In [5]:
function sayMyName(cmd, name) {
    console.log("my name is:", name);
}

In [6]:
%addmagic %name sayMyName

[ added magic: '%name' which will call function 'sayMyName' ]


In [7]:
%name mud

my name is: mud


## !exec
Lines that start with `!` will execute in the underlying shell.

In [8]:
!ls -laF

total 144
drwxr-xr-x   6 ampower  staff    192 Dec 29 07:57 ./
drwxr-xr-x  16 ampower  staff    512 Dec 28 15:48 ../
drwxr-xr-x   5 ampower  staff    160 Dec 28 14:41 .ipynb_checkpoints/
-rw-r--r--   1 ampower  staff   1615 Dec 28 10:34 Untitled.ipynb
-rw-r--r--   1 ampower  staff  10204 Dec 29 07:54 Untitled1.ipynb
-rw-r--r--   1 ampower  staff  56342 Dec 29 07:57 magicdemo.ipynb
[ process 'ls -laF' exited with code 0 ]


0

In [9]:
! ls

Untitled.ipynb
Untitled1.ipynb
magicdemo.ipynb
[ process 'ls' exited with code 0 ]


0

In [10]:
!npm ls

magicpatch@0.9.0 /Users/ampower/Projects/personal/magicpatch
├─┬ chai@4.2.0
│ ├── assertion-error@1.1.0
│ ├── check-error@1.0.2
│ ├─┬ deep-eql@3.0.1
│ │ └── type-detect@4.0.8 deduped
│ ├── get-func-name@2.0.0
│ ├── pathval@1.1.0
│ └── type-detect@4.0.8
├── chai-string@1.5.0
├── commander@6.2.1
├─┬ doctoc@2.0.0
│ ├─┬ @textlint/markdown-to-ast@6.1.7
│ │ ├── @textlint/ast-node-types@4.3.5
│ │ ├── debug@4.2.0 deduped
│ │ ├─┬ remark-frontmatter@1.3.3
│ │ │ ├─┬ fault@1.0.4
│ │ │ │ └── format@0.2.2
│ │ │ └── xtend@4.0.2
│ │ ├─┬ remark-parse@5.0.0
│ │ │ ├── collapse-white-space@1.0.6
│ │ │ ├── is-alphabetical@1.0.4
│ │ │ ├── is-decimal@1.0.4
│ │ │ ├── is-whitespace-character@1.0.4
│ │ │ ├── is-word-character@1.0.4
│ │ │ ├── markdown-escapes@1.0.4
│ │ │ ├─┬ parse-entities@1.2.2
│ │ │ │ ├── character-entities@1.2.4
│ │ │ │ ├── character-entities-legacy@1.1.4
│ │ │ │ ├── character-reference-invalid@1.1.4
│ │ │ │ ├─┬ is-alphanumerical@1.0.4
│ │ │ │ │ ├── is-alphabetical@1.0.4 deduped
│ │ │ │ │ └──

│ │ │ │ ├── @babel/types@7.12.12 deduped
│ │ │ │ └── lodash@4.17.20 deduped
│ │ │ ├─┬ @babel/helpers@7.12.5
│ │ │ │ ├── @babel/template@7.12.7 deduped
│ │ │ │ ├── @babel/traverse@7.12.12 deduped
│ │ │ │ └── @babel/types@7.12.12 deduped
│ │ │ ├── @babel/parser@7.12.11
│ │ │ ├─┬ @babel/template@7.12.7
│ │ │ │ ├── @babel/code-frame@7.12.11 deduped
│ │ │ │ ├── @babel/parser@7.12.11 deduped
│ │ │ │ └── @babel/types@7.12.12 deduped
│ │ │ ├─┬ @babel/traverse@7.12.12
│ │ │ │ ├── @babel/code-frame@7.12.11 deduped
│ │ │ │ ├── @babel/generator@7.12.11 deduped
│ │ │ │ ├─┬ @babel/helper-function-name@7.12.11
│ │ │ │ │ ├─┬ @babel/helper-get-function-arity@7.12.10
│ │ │ │ │ │ └── @babel/types@7.12.12 deduped
│ │ │ │ │ ├── @babel/template@7.12.7 deduped
│ │ │ │ │ └── @babel/types@7.12.12 deduped
│ │ │ │ ├── @babel/helper-split-export-declaration@7.12.11 deduped
│ │ │ │ ├── @babel/parser@7.12.11 deduped
│ │ │ │ ├── @babel/types@7.12.12 deduped
│ │ │ │ ├── debug@4.2.0 deduped
│ │ │ │ ├── globals@11.12.0

0

## Var Assignment
The output of %magic and !exec commands can be assigned to a variable for future reference.

In [11]:
let mod = %require ../test/helpers/testModule.js

[ loading /Users/ampower/Projects/personal/magicpatch/test/helpers/testModule.js ]


In [12]:
console.log("Module is:", mod);

Module is: { source: 'helpers/testModule.js', worked: true }


In [13]:
let dirListing = !ls -laF

ReferenceError: ret is not defined

##  {var} substitution

Using {var} will be replaced with the variable `var`. 

In [14]:
x = 42
%echo {x}

42


In [15]:
o = {
    beer: "yum"
}
%echo {o}

{ beer: 'yum' }


In [16]:
%echo x{x}o{o}

x42o[object Object]


## View Documentation
`?` before or after a magic will show you it's documentation

In [18]:
?%echo

__%echo Documentation:__

Usage: %echo &lt;string&gt;<br>
<br>
Write arguments to the standard output.<br>


__File:__ /Users/ampower/Projects/personal/magicpatch/lib/builtin.js

In [19]:
%echo?

__%echo Documentation:__

Usage: %echo &lt;string&gt;<br>
<br>
Write arguments to the standard output.<br>


__File:__ /Users/ampower/Projects/personal/magicpatch/lib/builtin.js

## View Code
`??` before or after a magic will show you the code for the magic.

In [20]:
??%echo

__%echo Source:__

``` js
function echo(cmd) {
    console.log(this.varSubst(this.line.substring(this.startArgs)));
}
```

__File:__ /Users/ampower/Projects/personal/magicpatch/lib/builtin.js

In [21]:
%echo??

__%echo Source:__

``` js
function echo(cmd) {
    console.log(this.varSubst(this.line.substring(this.startArgs)));
}
```

__File:__ /Users/ampower/Projects/personal/magicpatch/lib/builtin.js

## Input Caching
Previously run commands can be found in a few different ways. The last three commands are available in the global variables `_i`, `_ii`, and `_iii`. All previous commands can be found in the `In` or `_ih` arrays.

In [21]:
console.log(In[0])

var n = "bob";
console.log("hi", n);


In [22]:
console.log(In[1])

%echo hi there


In [23]:
console.log(_ih[1])

%echo hi there


In [24]:
console.log("last input:\n" + _i)
console.log("\nsecond to last input:\n" + _ii)
console.log("\ninput before that:\n" + _iii)

last input:
console.log(_ih[1])

second to last input:
console.log(In[1])

input before that:
console.log(In[0])


## Output Caching
The previous three output values are stored in `_`, `__`, and `___`.

In [25]:
21 * 2

42

In [26]:
"f" + "oo"

'foo'

In [27]:
Math.PI / 2

1.5707963267948966

In [28]:
console.log("last output:", _)
console.log("second to last output:", __)
console.log("output before that:", ___)

last output: 1.5707963267948966
second to last output: foo
output before that: 42


## Cell Magic
Cell magics do something special with the entire Jupyter cell. For example, this `%%script` cell magic turns the entire shell into a bash script.

In [29]:
%%script bash
for ((i = 0; i < 10; i++)); do
    echo "loop $i"
done

loop 0
loop 1
loop 2
loop 3
loop 4
loop 5
loop 6
loop 7
loop 8
loop 9
[ process 'bash /var/folders/6j/bk9lpc4n057831vsshxv0_j8t5f96f/T/tmp-59830-lTZ9PAyBhqm7' exited with code 0 ]


0

## stdmagic

A use a set of magics from a library

In [30]:
%require ../../stdmagic/index.js

[ loading /Users/ampower/Projects/personal/stdmagic/index.js ]


[
  { name: '%%bash', fn: [Function: stub], stub: true, standard: true },
  {
    name: '%%capture',
    fn: [Function: stub],
    stub: true,
    standard: true
  },
  { name: '%%debug', fn: [Function: stub], stub: true, standard: true },
  { name: '%%file', fn: [Function: stub], stub: true, standard: true },
  { name: '%%html', fn: [Function: stub], stub: true, standard: true },
  { name: '%%HTML', fn: [Function: stub], stub: true, standard: true },
  {
    name: '%%javascript',
    fn: [Function: stub],
    stub: true,
    standard: true
  },
  { name: '%%js', fn: [Function: stub], stub: true, standard: true },
  { name: '%%latex', fn: [Function: stub], stub: true, standard: true },
  {
    name: '%%markdown',
    fn: [Function: stub],
    stub: true,
    standard: true
  },
  { name: '%%perl', fn: [Function: stub], stub: true, standard: true },
  { name: '%%prun', fn: [Function: stub], stub: true, standard: true },
  { name: '%%pypy', fn: [Function: stub], stub: true, standard: tru

In [31]:
%lsmagic

ReferenceError: ret is not defined

In [32]:
%ls

ReferenceError: ret is not defined

In [33]:
%ls -laF

ReferenceError: ret is not defined

In [34]:
%who

ReferenceError: ret is not defined

In [35]:
%whos

ReferenceError: ret is not defined